In [1]:
!pip install findspark

In [2]:
import os
import sys
os.environ['SPARK_HOME']=r'/home/ritika/Downloads/spark-3.0.0-bin-hadoop2.7'

In [4]:
!pip install matplotlib
!pip install pandas

     |████████████████████████████████| 11.5 MB 241 kB/s eta 0:00:01
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
  Using cached kiwisolver-1.2.0-cp36-cp36m-manylinux1_x86_64.whl (88 kB)
  Using cached Pillow-7.2.0-cp36-cp36m-manylinux1_x86_64.whl (2.2 MB)


In [5]:
import random as pyrandom
import matplotlib
matplotlib.rc("image",cmap="gray")
import pandas as pd


In [6]:
from __future__ import print_function
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans, KMeansModel

In [7]:
if __name__ == "__main__":
    spark = SparkSession\
        .builder\
        .appName("kMeansWithSpark")\
        .getOrCreate()

In [8]:
# let's create a random dataset which we will use to write cluster algo with spark

In [9]:
df = spark.createDataFrame([[ 45.3, -17.5],
                              [ 42.4, -12.5],
                              [28., -23.9],
                              [29.5, -19.0],
                              [32.8, -18.84],
                            [23.8, -16.45],
                            [22.3, -15.54],
                            [12.8, -13.84]
                             ],
                              ["x", "y"])

In [10]:
df.show()

+----+------+
|   x|     y|
+----+------+
|45.3| -17.5|
|42.4| -12.5|
|28.0| -23.9|
|29.5| -19.0|
|32.8|-18.84|
|23.8|-16.45|
|22.3|-15.54|
|12.8|-13.84|
+----+------+



In [11]:
vecAssembler = VectorAssembler(inputCols=["x", "y"], outputCol="features")
new_df = vecAssembler.transform(df)
new_df.show()

+----+------+-------------+
|   x|     y|     features|
+----+------+-------------+
|45.3| -17.5| [45.3,-17.5]|
|42.4| -12.5| [42.4,-12.5]|
|28.0| -23.9| [28.0,-23.9]|
|29.5| -19.0| [29.5,-19.0]|
|32.8|-18.84|[32.8,-18.84]|
|23.8|-16.45|[23.8,-16.45]|
|22.3|-15.54|[22.3,-15.54]|
|12.8|-13.84|[12.8,-13.84]|
+----+------+-------------+



In [12]:
# Train a k-means model.
kmeans = KMeans().setK(2).setSeed(1)
model = kmeans.fit(new_df.select('features'))

In [13]:
# Make predictions
predictions = model.transform(new_df)
predictions.show()

+----+------+-------------+----------+
|   x|     y|     features|prediction|
+----+------+-------------+----------+
|45.3| -17.5| [45.3,-17.5]|         1|
|42.4| -12.5| [42.4,-12.5]|         1|
|28.0| -23.9| [28.0,-23.9]|         0|
|29.5| -19.0| [29.5,-19.0]|         0|
|32.8|-18.84|[32.8,-18.84]|         1|
|23.8|-16.45|[23.8,-16.45]|         0|
|22.3|-15.54|[22.3,-15.54]|         0|
|12.8|-13.84|[12.8,-13.84]|         0|
+----+------+-------------+----------+



In [14]:
spark.stop()